# Dark Sky API (Weather)

Answer the following questions using the [Dark Sky API](https://darksky.net/dev/). I've added three cells for each question but you're free to use more or less! Hold `Shift` and hit `Enter` to run a cell, and use the `+` on the top left to add a new cell to a notebook.

If you spend some time reading the documentation your life will probably be a little bit easier!

## 0) Import any libraries you might need

- *Tip: We're going to be downloading things from the internet, so we probably need `requests`.*

In [1]:
import requests

## 1) Make a request from the Dark Sky API for where you were born (or lived, or want to visit!).

- *Tip: The URL we used in class was for a place near San Francisco. What was the format of the endpoint that made this happen?*
- *Tip: Save the URL as a separate variable, and be sure to not have `[` and `]` inside.*
- *Tip: How is north vs. south and east vs. west latitude/longitude represented? Is it the normal North/South/East/West?*
- *Tip: You know it's JSON, but Python doesn't! Make sure you aren't trying to deal with plain text.* 
- *Tip: Once you've imported the JSON into a variable, check the timezone's name to make sure it seems like it got the right part of the world!*

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

API_KEY = os.getenv("DARKSKY_API_KEY")

In [3]:
response = requests.get(f'https://api.darksky.net/forecast/{API_KEY}/22.3193,114.1694')
hk = response.json()

## 2) What's the current wind speed? How much warmer does it feel than it actually is?

- *Tip: You can do this by browsing through the dictionaries, but it might be easier to read the documentation*
- *Tip: For the second half: it **is** one temperature, and it **feels** a different temperature. Calculate the difference.*

In [4]:
print(f"The current wind speed is {hk['currently']['windSpeed']} mph.")

The current wind speed is 12.86 mph.


In [5]:
print(f"It feels {round(hk['currently']['apparentTemperature'] - hk['currently']['temperature'],2)} °F warmer than it actually is.")

It feels 1.62 °F warmer than it actually is.


## 3) Today's forecast is the FIRST daily forecast. For the place you decided on above, how much of the moon is currently visible?

- *Tip: This does **not** use the `currently` key.*
- *Tip: It might be helpful to save the forecast into its own variable*

In [6]:
today = hk['daily']['data'][0]
moon = today['moonPhase']

In [7]:
if moon == 0:
    print("It is a new moon.")
elif moon < 0.25:
    print("It is a waxing crescent moon.")
elif moon == 0.25:
    print("It is a first quarter moon.")
elif moon < 0.5:
    print("It is a waxing gibbous moon.")
elif moon == 0.5:
    print("It is a full moon.")
elif moon < 0.75:
    print("It is a waning gibbous moon.")
elif moon == 0.75:
    print("It is a last quarter moon.")
else:
    print("It is a waning crescent moon.")

It is a waxing gibbous moon.


## 4) What's the difference between the high and low temperatures for today?

In [8]:
print(today["temperatureHigh"] - today["temperatureLow"])

3.8299999999999983


## 5) Go through the daily forecasts, printing out the next week's worth of predictions.

I'd like to know the **high temperature** for each day, and whether it's **hot, warm, or cold** (based on what temperatures you think are hot, warm or cold).

- *Tip: Is it returning F or C? You can change them by adding `?units=us` or `?units=si` onto the end of the URL.*
- *Tip: You'll need to use an `if` statement to say whether it is hot, warm or cold.*

In [12]:
for day in hk['daily']['data']:
    if day["temperatureHigh"] <= 59:
        print(f'The high temperture is {day["temperatureHigh"]} °F and it is cold.')
    elif day["temperatureHigh"] >= 82:
        print(f'The high temperture is {day["temperatureHigh"]} °F and it is hot.')
    else:
        print(f'The high temperture is {day["temperatureHigh"]} °F and it is warm.')

The high temperture is 79.23 °F and it is warm.
The high temperture is 83.11 °F and it is hot.
The high temperture is 82.18 °F and it is hot.
The high temperture is 82.28 °F and it is hot.
The high temperture is 82.02 °F and it is hot.
The high temperture is 80.16 °F and it is warm.
The high temperture is 79.56 °F and it is warm.
The high temperture is 79.49 °F and it is warm.


# 6) What will be the hottest day in the next week? What is the high temperature on that day?

It's okay if your result looks like `1453363200`. If you want to be extra-fancy, though, format it as a **real date** - the weird number is called a **UNIX timestamp**, which might help in googling.

- *Tip: It's okay to skip this if it seems too difficult!*

In [10]:
hottest = 0
for day in hk['daily']['data']:
    if day["temperatureHigh"] > hottest:
        hottest = day["temperatureHigh"]

In [11]:
for day in hk['daily']['data']:
    if day["temperatureHigh"] == hottest:
        import datetime
        print(f'{datetime.datetime.utcfromtimestamp(day["time"])} UTC will be the hottest day in the next week and the high temperature on that day is {hottest} °F.')  

2019-06-11 16:00:00 UTC will be the hottest day in the next week and the high temperature on that day is 83.11 °F.


## 7) What's the weather looking like for the next 24+ hours in Miami, Florida?

I'd like to know the temperature for every hour, and if it's going to have cloud cover of more than 50% say "{temperature} and cloudy" instead of just the temperature. 


- *Tip: Be sure to remember how latitude and longitude are done with positive and negative numbers*
- *Tip: How do you represent "50%" for cloud cover?*

In [12]:
response1 = requests.get(f'https://api.darksky.net/forecast/{API_KEY}/25.7617,-80.1918')
miami = response1.json()

In [13]:
hours = miami['hourly']['data']
hours24 = [hour for hour in hours if hours[0]['time'] < hour['time'] < hours[25]['time']]
for hour in hours24:
    if hour['cloudCover'] > 0.5:
        print(f"{hour['temperature']} and cloudy")
    else:
        print(hour['temperature'])

79.46
79.98
80.24
79.59
79.24
79.5
80.27 and cloudy
82.09 and cloudy
84.72
86.76
88.16
89.16 and cloudy
89.77 and cloudy
91.42 and cloudy
91.39 and cloudy
90.41 and cloudy
89.11 and cloudy
87.76 and cloudy
86.29 and cloudy
85.09 and cloudy
84.28 and cloudy
83.72
82.9
82.23


# 8) For the next 24-ish hours in Miami, what percent of the time is the temperature above 85 degrees?

If they give you 48 hours instead of 24, that's okay.

- *Tip: You might want to read up on [looping patterns](http://jonathansoma.com/lede/foundations-2017/classes/data%20structures/looping-patterns/)*

In [14]:
c_hour = 0
for hour in hours24:
    if hour['temperature'] > 85:
        c_hour = c_hour + 1
cloudper = round(c_hour / 24 * 100)
print(f"For the next 24 hours in Miama, the temperature is above 85 degrees for {cloudper} percent of the time.")

For the next 24 hours in Miama, the temperature is above 85 degrees for 46 percent of the time.


## 9) What was the temperature in Central Park on Christmas Day, 2012? How about 2007? 2005? How far back does the API allow you to go?

- *Tip: You'll need to use UNIX time, which is the number of seconds since January 1, 1970. Google can help you convert a normal date!*
- *Tip: You'll want to use Forecast.io's "time machine" API, [documentation here](https://darksky.net/dev/docs/time-machine)*

In [15]:
park12 = requests.get(f'https://api.darksky.net/forecast/{API_KEY}/40.7829,-73.9654,1356436800').json()
park07 = requests.get(f'https://api.darksky.net/forecast/{API_KEY}/40.7829,-73.9654,1198584000').json()
park05 = requests.get(f'https://api.darksky.net/forecast/{API_KEY}/40.7829,-73.9654,1135512000').json()
print(f"The temperature in Central Park on Christmas Day was {park12['daily']['data'][0]['temperatureLow']} to {park12['daily']['data'][0]['temperatureHigh']} °F in 2012, {park07['daily']['data'][0]['temperatureLow']} to {park07['daily']['data'][0]['temperatureHigh']} °F in 2007 and {park05['daily']['data'][0]['temperatureLow']} to {park05['daily']['data'][0]['temperatureHigh']} °F in 2005.")

The temperature in Central Park on Christmas Day was 27.78 to 40.28 °F in 2012, 29.55 to 42.33 °F in 2007 and 45.66 to 45.73 °F in 2005.


In [16]:
park00 = requests.get(f'https://api.darksky.net/forecast/{API_KEY}/40.7829,-73.9654,-1041364799').json()
print(f"The earliest date that the API allows users to go to is around {datetime.datetime.fromtimestamp(-1041364799)}.")

The earliest date that the API allows users to go to is around 1936-12-31 23:00:01.
